In [2]:
import os
import pandas as pd
import swat
import sys

In [3]:
os.environ["CAS_CLIENT_SSL_CA_LIST"] = "/opt/sas/viya/config/etc/SASSecurityCertificateFramework/cacerts/trustedcerts.pem"
cashost='frasepviya35smp.cloud.com'
casport=5570
sess = swat.CAS(cashost, casport)

In [4]:
sess.loadactionset('datastep')
sess.loadactionset('transpose')

NOTE: Added action set 'datastep'.
NOTE: Added action set 'transpose'.


[actionset]

 'transpose'

+ Elapsed: 0.0192s, user: 0.00067s, sys: 0.000367s, mem: 0.204mb

## Transpose automatically at scale all interesting measure from line to column

In [13]:
results=sess.transpose.transpose(                                             
   table={"name":"SENSOR_DATA_ALL","caslib":"PUBLIC",
          "groupBy":[
               {"name":"sitecode"},
               {"name":"vehicle_id"},
               {"name":"machine_name"},
               {"name":"Fluid"},
               {"name":"step_name"},
               {"name":"stepid"},
               {"name":"sourcetimestamp"}]},
   transpose={"sensor_value"},
   let="True",
   id={"name":"sensor_name"},
   casOut={"name":"SENSOR_DATA_ABT", "replace":True})

NOTE: There were 5907527 rows read from the table SENSOR_DATA_ALL.
NOTE: An additional 211446 warnings were suppressed.
NOTE: The table SENSOR_DATA_ABT has 5338504 rows and 11 columns.


In [26]:
sess.table.tableDetails(name="SENSOR_DATA_ABT")
#sess.table.tableDetails(level="partition",name="SENSOR_DATA_ABT")

,Node,Blocks,Active,Rows,IndexSize,DataSize,VardataSize,CompressedSize,CompressionRatio,Mapped,MappedMemory,Unmapped,UnmappedMemory,Allocated,AllocatedMemory,DeletedRows
0,ALL,768,768,5338504,0,804541248,163920768,0,0,754,790259984,0,0,14,14371384,0


## Promote resulting in-memory table as public to share it for reporting and analysis and serialize it to disk for resilience

In [7]:
#sess.droptable(name="SENSOR_DATA_ABT", caslib="PUBLIC", quiet="TRUE")
#sess.promote(name="SENSOR_DATA_ABT", targetCaslib="PUBLIC")
#sess.table.save(table=dict(name='SENSOR_DATA_ABT', caslib='PUBLIC'), caslib='PUBLIC', name='SENSOR_DATA_ABT.sashdat', replace=True)

NOTE: Cloud Analytic Services promoted table SENSOR_DATA_ABT in caslib CASUSER(viyademo01) to table SENSOR_DATA_ABT in caslib PUBLIC.
NOTE: Cloud Analytic Services saved the file SENSOR_DATA_ABT.sashdat in caslib Public.


[caslib]

 'Public'

[name]

 'SENSOR_DATA_ABT.sashdat'

+ Elapsed: 9.56s, user: 0.355s, sys: 0.703s, mem: 17mb

## Impute not changing sensor measure with last known in the transposed ABT

In [27]:
dscode='''
DATA casuser.SENSOR_DATA_ABT_V2(drop=cur_ActLoadingPressure cur_ActResidualPressure cur_ActVacuum);
        SET casuser.SENSOR_DATA_ABT;
        by machine_name Fluid step_name sourcetimestamp;
        retain cur_ActLoadingPressure cur_ActResidualPressure cur_ActVacuum;

        if missing(ActLoadingPressure) then ActLoadingPressure = cur_ActLoadingPressure;
        else cur_ActLoadingPressure = ActLoadingPressure;

        if missing(ActResidualPressure) then ActResidualPressure = cur_ActResidualPressure;
        else cur_ActResidualPressure = ActResidualPressure;

        if missing(ActVacuum) then ActVacuum = cur_ActVacuum;
        else cur_ActVacuum = ActVacuum;
RUN;''';

sess.dataStep.runCode(dscode)

[InputCasTables]

                 casLib             Name     Rows  Columns  \
 0  CASUSER(viyademo01)  SENSOR_DATA_ABT  5338504       11   
 
                                             casTable  
 0  CASTable('SENSOR_DATA_ABT', caslib='CASUSER(vi...  

[OutputCasTables]

                 casLib                Name     Rows  Columns  Append Promoted  \
 0  CASUSER(viyademo01)  SENSOR_DATA_ABT_V2  5338504       11     NaN        N   
 
                                             casTable  
 0  CASTable('SENSOR_DATA_ABT_V2', caslib='CASUSER...  

+ Elapsed: 43.4s, user: 80.8s, sys: 0.458s, mem: 22.5mb

## Promote resulting in-memory table as public to share it for reporting and analysis and serialize it to disk for resilience

In [28]:
sess.table.droptable(name="SENSOR_DATA_ABT", caslib="PUBLIC", quiet="TRUE")
sess.table.promote(name="SENSOR_DATA_ABT_V2",target="SENSOR_DATA_ABT", targetLib="PUBLIC", )
sess.table.save(table=dict(name='SENSOR_DATA_ABT', caslib='PUBLIC'), caslib='PUBLIC', name='SENSOR_DATA_ABT.sashdat', replace=True)

NOTE: Cloud Analytic Services promoted table SENSOR_DATA_ABT_V2 in caslib CASUSER(viyademo01) to table SENSOR_DATA_ABT in caslib PUBLIC.
NOTE: Cloud Analytic Services saved the file SENSOR_DATA_ABT.sashdat in caslib Public.


[caslib]

 'Public'

[name]

 'SENSOR_DATA_ABT.sashdat'

+ Elapsed: 11.3s, user: 1.41s, mem: 17mb

In [29]:
sess.terminate()